In [1]:
# --- Installation ---
# These commands should be run in your terminal or a Jupyter cell (with a '!')
# pip install pandas spacy requests geopy
# python -m spacy download en_core_web_sm

# Functional Implementation

In [2]:
import re
import pandas as pd
import requests
import spacy
import sqlite3
from geopy.geocoders import Nominatim

# --- 1. INITIAL SETUP AND CONFIGURATION ---

nlp = spacy.load('en_core_web_sm')
geolocator = Nominatim(user_agent="weather_chatbot_app")

# --- DataFrames for mapping user input to bot actions ---
expressions_df = pd.DataFrame({
    'expression': ['temperature|temp', 'sunny|cloudy|conditions|weather', 'wind speed|wind', 'rain', 'snowfall', 'snow depth', 'hi|hello|hey'],
    'intent': ['ask_temperature', 'ask_weather_conditions', 'ask_wind_speed', 'ask_rain', 'ask_snowfall', 'ask_snow_depth', 'greet']
})

intents_df = pd.DataFrame({
    'intent': ['ask_temperature', 'ask_weather_conditions', 'ask_wind_speed', 'ask_rain', 'ask_snowfall', 'ask_snow_depth', 'greet'],
    'action': ['get_temperature', 'get_weather_conditions', 'get_wind_speed', 'get_rain', 'get_snowfall', 'get_snow_depth', 'greet_user']
})

responses_df = pd.DataFrame({
    'action': ['get_temperature', 'get_weather_conditions', 'get_wind_speed', 'get_rain', 'get_snowfall', 'get_snow_depth', 'greet_user'],
    'response_template': [
        "The temperature is {temperature}.", "The current condition is: {weather_conditions}.",
        "The wind speed is {wind_speed}.", "The rainfall is {rain}.",
        "The snowfall is {snowfall}.", "The snow depth is {snow_depth}.",
        "Hello! How can I help you with the weather today?"
    ]
})

# --- 2. DATA FETCHING (UNIFIED FUNCTION) ---

def get_weather_info(city, use_mock=True):
    """
    Fetches weather data by first checking a local database (cache).
    If not found, it fetches new data based on the `use_mock` flag.
    """
    conn = None
    try:
        conn = sqlite3.connect('weather_database.db')
        cursor = conn.cursor()
        cursor.execute("SELECT * FROM weather_info WHERE city=?", (city,))
        cached_result = cursor.fetchone()

        if cached_result:
            return {'temperature': cached_result[1], 'weather_conditions': cached_result[2], 'wind_speed': cached_result[3], 'rain': cached_result[4], 'snowfall': cached_result[5], 'snow_depth': cached_result[6]}

        weather_info = {}
        if use_mock:
            weather_info = {'temperature': '25°C', 'weather_conditions': 'Sunny', 'wind_speed': '15 km/h', 'rain': '0 mm', 'snowfall': '0 mm', 'snow_depth': '0 cm'}
        else:
            try:
                location = geolocator.geocode(city)
                if not location: return {}
                lat, lon = location.latitude, location.longitude
                api_url = f"https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&current=temperature_2m,weather_code,rain,snowfall,wind_speed_10m&temperature_unit=celsius&wind_speed_unit=kmh"
                response = requests.get(api_url)
                response.raise_for_status()
                data = response.json()['current']
                units = response.json()['current_units']
                weather_info = {
                    'temperature': f"{data.get('temperature_2m', 'N/A')}{units.get('temperature_2m', '')}",
                    'weather_conditions': f"{data.get('weather_code', 'N/A')}",
                    'wind_speed': f"{data.get('wind_speed_10m', 'N/A')}{units.get('wind_speed_10m', '')}",
                    'rain': f"{data.get('rain', 'N/A')}{units.get('rain', '')}",
                    'snowfall': f"{data.get('snowfall', 'N/A')}{units.get('snowfall', '')}",
                    'snow_depth': 'N/A'
                }
            except requests.RequestException as e:
                print(f"API request error: {e}")
                return {}

        if weather_info:
            cursor.execute('''INSERT OR REPLACE INTO weather_info VALUES (?, ?, ?, ?, ?, ?, ?)''', (city, weather_info['temperature'], weather_info['weather_conditions'], weather_info['wind_speed'], weather_info['rain'], weather_info['snowfall'], weather_info['snow_depth']))
            conn.commit()
        return weather_info
    except sqlite3.Error as e:
        print(f"Database error: {e}")
        return {}
    finally:
        if conn: conn.close()

# --- 3. CORE CHATBOT COMPONENTS ---

def extract_entities(text):
    """Interprets the user's message to find intents and city names."""
    entities = {}
    # Iterate over expressions DataFrame
    for index, row in expressions_df.iterrows():
        expression_match = re.search(fr'\b(?:{row["expression"]})\b', text, re.IGNORECASE)
        if expression_match:
            entities[row['intent']] = True
    doc = nlp(text)
    cities = [entity.text for entity in doc.ents if entity.label_ == 'GPE']
    entities['cities'] = cities
    return entities


def manage_dialog(query, use_mock):
    """
    Manages the conversation flow, handling greetings and prompting for the missing information.
    """
    entities = extract_entities(query)
    actions = intents_df[intents_df['intent'].isin(entities.keys())]['action'].tolist()
    cities = entities.get('cities', [])
    
    # Handle greetings separately first
    actions = handle_greet_user(actions)

    # If it was *only* a greeting.
    while not actions:
        user_query = input("Bot: I can provide the temperature and weather & wind & Snow conditions.\nWhat are you interested in? (Enter 'q' to quit)\nYou: ")
        if user_query.lower() == 'q':
            print("Goodbye!")
            return {}
        # We re-evaluate the whole query.
        entities = extract_entities(user_query)
        actions = intents_df[intents_df['intent'].isin(entities.keys())]['action'].tolist()
        actions = handle_greet_user(actions)
        cities = entities.get('cities', [])

    # If it were actions without cities 
    while not cities:
        cities = prompt_for_city()
        if not cities: 
            return {}

    # Once we have both actions and cities, generate the response.
    all_responses = {}
    for city in cities:
        weather_info = get_weather_info(city, use_mock=use_mock)
        if weather_info:
            all_responses[city] = [generate_response(action, weather_info) for action in actions]
    return all_responses

# Function to get the city if it is missing in the user query
def prompt_for_city():
    while True:
        user_city = input("Bot: For which city would you like the weather forecast? (Enter 'q' to quit)\nYou: ")
        if user_city.lower() == 'q':
            print("Goodbye!")
            return []
        doc = nlp(user_city)
        cities = [entity.text for entity in doc.ents if entity.label_ == 'GPE']
        if cities:
            return cities
        print("Please provide a valid city name.")

# Function to handle 'greet_user' action
def handle_greet_user(user_actions):
    if 'greet_user' in user_actions:
        print(responses_df.loc[responses_df['action'] == 'greet_user', 'response_template'].values[0])
        user_actions.remove('greet_user')
    return user_actions

def generate_response(action, weather_info):
    """Constructs the final message strings to send to the user."""
    if action in responses_df['action'].values:
        template = responses_df.loc[responses_df['action'] == action, 'response_template'].values[0]
        return template.format(**weather_info)
    return "I'm sorry, I don't understand the request."
    
# --- 4. DATABASE SETUP ---
def create_weather_database():
    """Creates the SQLite database and table if they don't exist."""
    conn = None
    try:
        conn = sqlite3.connect('weather_database.db')
        cursor = conn.cursor()
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS weather_info (
                city TEXT PRIMARY KEY, temperature TEXT, weather_conditions TEXT,
                wind_speed TEXT, rain TEXT, snowfall TEXT, snow_depth TEXT
            )
        ''')
        conn.commit()
    except sqlite3.Error as e:
        print(f"Error creating database: {e}")
    finally:
        if conn: conn.close()

# --- 5. MAIN CHATBOT LOOP ---
if __name__ == "__main__":
    # ---!!! CHATBOT MODE CONTROL !!!---
    USE_MOCK_DATA = True

    create_weather_database()
    print("Weather Chatbot is running!")
    print(f"** Current Mode: {'MOCK' if USE_MOCK_DATA else 'LIVE API'} **")
    print("-" * 30)

    while True:
        user_query = input("Bot: I can provide the temperature and weather & wind & Snow conditions.\nWhat are you interested in? (Enter 'q' to quit)\nYou: ")
        if user_query.lower() == 'q':
            print("Bot: Goodbye!")
            print("*" * 30)
            break
        responses = manage_dialog(user_query, use_mock=USE_MOCK_DATA)
        if responses:
            for city, response_list in responses.items():
                print(f"For {city}:")
                for response in response_list:
                    print(response)
            print("*" * 30)
            print()
        else:
            continue

Weather Chatbot is running!
** Current Mode: MOCK **
------------------------------


Bot: I can provide the temperature and weather & wind & Snow conditions.
What are you interested in? (Enter 'q' to quit)
You:  hey


Hello! How can I help you with the weather today?


Bot: I can provide the temperature and weather & wind & Snow conditions.
What are you interested in? (Enter 'q' to quit)
You:  temp and wind speed for berlin and cairo


For berlin:
The temperature is 30 °C.
The wind speed is 50 Km/h.
For cairo:
The temperature is 30 °C.
The wind speed is 50 Km/h.
******************************



Bot: I can provide the temperature and weather & wind & Snow conditions.
What are you interested in? (Enter 'q' to quit)
You:  q


Bot: Goodbye!
******************************


# Object-Oriented/Class-Based Implementation

In [3]:
import re
import pandas as pd
import requests
import spacy
import sqlite3
from geopy.geocoders import Nominatim

# --- 1. INITIAL SETUP AND CONFIGURATION ---
nlp = spacy.load('en_core_web_sm')
geolocator = Nominatim(user_agent="weather_chatbot_app_classes")

# --- Global DataFrames for chatbot configuration ---
expressions_df = pd.DataFrame({
    'expression': ['temperature|temp', 'sunny|cloudy|conditions', 'wind speed', 'rain', 'snowfall', 'snow depth', 'hi|hello|hey'],
    'intent': ['ask_temperature', 'ask_weather_conditions', 'ask_wind_speed', 'ask_rain', 'ask_snowfall', 'ask_snow_depth', 'greet']
})
intents_df = pd.DataFrame({
    'intent': ['ask_temperature', 'ask_weather_conditions', 'ask_wind_speed', 'ask_rain', 'ask_snowfall', 'ask_snow_depth', 'greet'],
    'action': ['get_temperature', 'get_weather_conditions', 'get_wind_speed', 'get_rain', 'get_snowfall', 'get_snow_depth', 'greet_user']
})
responses_df = pd.DataFrame({
    'action': ['get_temperature', 'get_weather_conditions', 'get_wind_speed', 'get_rain', 'get_snowfall', 'get_snow_depth', 'greet_user'],
    'response_template': [
        "The temperature is {temperature}.", "The current condition is: {weather_conditions}.",
        "The wind speed is {wind_speed}.", "The rainfall is {rain}.",
        "The snowfall is {snowfall}.", "The snow depth is {snow_depth}.",
        "Hello! How can I help you with the weather today?"
    ]
})

# --- 2. CORE CLASSES ---
class WeatherData:
    """A simple class to hold weather data attributes."""
    def __init__(self, temperature, weather_conditions, wind_speed, rain, snowfall, snow_depth):
        self.temperature = temperature
        self.weather_conditions = weather_conditions
        self.wind_speed = wind_speed
        self.rain = rain
        self.snowfall = snowfall
        self.snow_depth = snow_depth

class WeatherDatabase:
    """Handles all data storage and retrieval, including caching and API calls."""
    def __init__(self, db_path):
        self.db_path = db_path
        self._create_table()

    def _create_table(self):
        """Private method to create the database table if it doesn't exist."""
        conn = None
        try:
            conn = sqlite3.connect(self.db_path)
            cursor = conn.cursor()
            cursor.execute('''
                CREATE TABLE IF NOT EXISTS weather_info (
                    city TEXT PRIMARY KEY, temperature TEXT, weather_conditions TEXT,
                    wind_speed TEXT, rain TEXT, snowfall TEXT, snow_depth TEXT
                )
            ''')
            conn.commit()
        except sqlite3.Error as e:
            print(f"Database creation error: {e}")
        finally:
            if conn:
                conn.close()

    def get_weather(self, city, use_mock=True):
        """Main method to get weather data from cache or a fresh source."""
        conn = None
        try:
            conn = sqlite3.connect(self.db_path)
            cursor = conn.cursor()
            cursor.execute("SELECT * FROM weather_info WHERE city=?", (city,))
            cached_result = cursor.fetchone()

            if cached_result:
                return WeatherData(cached_result[1], cached_result[2], cached_result[3], cached_result[4], cached_result[5], cached_result[6])

            # If not in cache, fetch new data
            weather_info = self.fetch_from_mock_source() if use_mock else self.fetch_from_live_api(city)

            if weather_info:
                cursor.execute('''
                    INSERT OR REPLACE INTO weather_info (city, temperature, weather_conditions, wind_speed, rain, snowfall, snow_depth)
                    VALUES (?, ?, ?, ?, ?, ?, ?)
                ''', (city, weather_info.temperature, weather_info.weather_conditions,
                      weather_info.wind_speed, weather_info.rain, weather_info.snowfall,
                      weather_info.snow_depth))
                conn.commit()
            return weather_info
        except sqlite3.Error as e:
            print(f"Database error: {e}")
            return None
        finally:
            if conn:
                conn.close()

    def fetch_from_mock_source(self):
        """Method to return hardcoded data for testing."""
        return WeatherData('25°C', 'Sunny', '15 km/h', '0 mm', '0 mm', '0 cm')

    def fetch_from_live_api(self, city):
        """Method to fetch real data from the Open-Meteo API."""
        try:
            location = geolocator.geocode(city)
            if not location:
                print(f"Could not geocode city: {city}")
                return None
            lat, lon = location.latitude, location.longitude
            api_url = f"https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&current=temperature_2m,weather_code,rain,snowfall,wind_speed_10m&temperature_unit=celsius&wind_speed_unit=kmh"
            response = requests.get(api_url)
            response.raise_for_status()
            data = response.json()['current']
            units = response.json()['current_units']

            return WeatherData(
                temperature=f"{data.get('temperature_2m', 'N/A')}{units.get('temperature_2m', '')}",
                weather_conditions=data.get('weather_code', 'N/A'),
                wind_speed=f"{data.get('wind_speed_10m', 'N/A')}{units.get('wind_speed_10m', '')}",
                rain=f"{data.get('rain', 'N/A')}{units.get('rain', '')}",
                snowfall=f"{data.get('snowfall', 'N/A')}{units.get('snowfall', '')}",
                snow_depth='N/A'
            )
        except requests.RequestException as e:
            print(f"API request error: {e}")
            return None

class NLUComponent:
    """Handles Natural Language Understanding to extract intents and entities."""
    def __init__(self, expressions_df, intents_df):
        self.expressions_df = expressions_df
        self.intents_df = intents_df

    def process(self, text):
        """Extracts actions and city names from a user's query."""
        intents = {row["intent"] for _, row in self.expressions_df.iterrows() if re.search(fr'\b({row["expression"]})\b', text, re.IGNORECASE)}
        doc = nlp(text)
        cities = [ent.text for ent in doc.ents if ent.label_ == 'GPE']
        actions = self.intents_df[self.intents_df['intent'].isin(intents)]['action'].tolist()
        return actions, cities

class ResponseGenerator:
    """Generates the final text responses for the user."""
    def __init__(self, responses_df):
        self.responses_df = responses_df

    def generate(self, actions, weather_info):
        """Formats response strings based on the requested actions and provided data."""
        if not weather_info:
            return ["Sorry, I couldn't retrieve weather information for this city."]
        response_list = []
        for action in actions:
            if action in self.responses_df['action'].values:
                template = self.responses_df.loc[self.responses_df['action'] == action, 'response_template'].values[0]
                response_list.append(template.format(**weather_info.__dict__))
        return response_list

class DialogManager:
    """Manages the conversation flow between the user and the chatbot."""
    def __init__(self, nlu, responses_df):
        self.nlu = nlu
        self.responses_df = responses_df

    def manage(self, query):
        """Orchestrates the conversation turn, prompting for missing info."""
        actions, cities = self.nlu.process(query)

        if 'greet_user' in actions:
            print("Bot: " + self.responses_df.loc[self.responses_df['action'] == 'greet_user', 'response_template'].values[0])
            actions.remove('greet_user')
            if not actions and not cities:
                return None, None

        while not actions:
            clarification = input("Bot: I can provide temperature, conditions, and wind speed. What are you interested in? (Enter 'q' to quit)\nYou: ")
            if clarification.lower() == 'q': return None, None
            actions, new_cities = self.nlu.process(clarification)
            if new_cities: cities = new_cities

        while not cities:
            user_city = input("Bot: For which city would you like the weather forecast? (Enter 'q' to quit)\nYou: ")
            if user_city.lower() == 'q': return None, None
            _, cities = self.nlu.process(user_city)

        return actions, cities

# --- 3. MAIN CHATBOT EXECUTION ---

if __name__ == "__main__":
    # ---!!! CHATBOT MODE CONTROL !!!---
    USE_MOCK_DATA = True

    # --- Instantiate all components ---
    weather_db = WeatherDatabase('weather_database.db')
    nlu = NLUComponent(expressions_df, intents_df)
    response_gen = ResponseGenerator(responses_df)
    dialog_manager = DialogManager(nlu, responses_df)

    print("Weather Chatbot is running!")
    print(f"** Current Mode: {'MOCK' if USE_MOCK_DATA else 'LIVE API'} **")
    print("-" * 30)

    while True:
        user_query = input("Bot: How can I help you with the weather today? (Enter 'q' to quit)\nYou: ")
        if user_query.lower() == 'q':
            print("Bot: Goodbye!")
            break

        actions, cities = dialog_manager.manage(user_query)

        if actions and cities:
            for city in cities:
                weather_info = weather_db.get_weather(city, use_mock=USE_MOCK_DATA)
                responses = response_gen.generate(actions, weather_info)

                print(f"\nBot (for {city}):")
                for res in responses:
                    print(f"- {res}")
        print("-" * 30)

Weather Chatbot is running!
** Current Mode: MOCK **
------------------------------


Bot: How can I help you with the weather today? (Enter 'q' to quit)
You:  hey


Bot: Hello! How can I help you with the weather today?
------------------------------


Bot: How can I help you with the weather today? (Enter 'q' to quit)
You:  temp and wind speed for berlin and cairo



Bot (for berlin):
- The temperature is 30 °C.
- The wind speed is 50 Km/h.

Bot (for cairo):
- The temperature is 30 °C.
- The wind speed is 50 Km/h.
------------------------------


Bot: How can I help you with the weather today? (Enter 'q' to quit)
You:  q


Bot: Goodbye!
